In [1]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,1))
x = torch.rand(size=(2,4))
net(x)

F:\programData\conda\env\ChatGLM2-6B\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
F:\programData\conda\env\ChatGLM2-6B\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
F:\programData\conda\env\ChatGLM2-6B\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


tensor([[0.2525],
        [0.2997]], grad_fn=<AddmmBackward0>)

In [2]:
# 5.2.1. 参数访问
print(net[2].state_dict())

OrderedDict([('weight', tensor([[ 0.2420,  0.0767, -0.3450, -0.2489,  0.1084,  0.1857, -0.2361,  0.2070]])), ('bias', tensor([0.2874]))])


In [4]:
# 5.2.1.1. 目标参数
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.2874], requires_grad=True)
tensor([0.2874])


In [5]:
net[2].weight.grad == None

True

In [6]:
# 5.2.1.2. 一次性访问所有参数
print(*[(name,param.shape) for name,param in net[0].named_parameters()])
print(*[(name,param.shape) for name,param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [7]:
net.state_dict()['2.bias'].data

tensor([0.2874])

In [9]:
net.state_dict()

OrderedDict([('0.weight',
              tensor([[-0.3605, -0.4577,  0.3131,  0.2453],
                      [ 0.2417,  0.2608, -0.2790, -0.4957],
                      [-0.0833,  0.2403, -0.1582,  0.1253],
                      [-0.0920,  0.2802,  0.2884, -0.4605],
                      [ 0.1369, -0.4981, -0.3670,  0.0551],
                      [ 0.2145, -0.0649, -0.1354, -0.3796],
                      [-0.2478,  0.0305,  0.2289, -0.1056],
                      [ 0.4535,  0.1551, -0.1220, -0.1581]])),
             ('0.bias',
              tensor([-0.4648, -0.3364,  0.1526,  0.0738, -0.1145,  0.0186, -0.2801,  0.3195])),
             ('2.weight',
              tensor([[ 0.2420,  0.0767, -0.3450, -0.2489,  0.1084,  0.1857, -0.2361,  0.2070]])),
             ('2.bias', tensor([0.2874]))])

In [10]:
# 5.2.1.3. 从嵌套块收集参数
def block1():
    return nn.Sequential(nn.Linear(4,8)
                         ,nn.ReLU()
                         ,nn.Linear(8,4)
                         ,nn.ReLU())
def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}',block1())
    return net

rgent = nn.Sequential(block2(),nn.Linear(4,1))
rgent(x)

tensor([[-0.2981],
        [-0.2978]], grad_fn=<AddmmBackward0>)

In [11]:
print(rgent)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [12]:
rgent[0][1][0].bias.data

tensor([ 0.1380,  0.3023,  0.1194,  0.2451,  0.2825,  0.1974, -0.3113, -0.4589])

In [ ]:
# 5.2.2. 参数初始化

In [14]:
#5.2.2.1. 内置初始化
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight,mean=0,std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0],net[0].bias.data[0]

(tensor([ 0.0101, -0.0114, -0.0041,  0.0071]), tensor(0.))

In [16]:
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight,42)

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data)
print(net[2].weight.data)



tensor([[-0.3558,  0.3792, -0.1647,  0.5043],
        [-0.0053,  0.0374, -0.2313, -0.2861],
        [-0.0391, -0.6666,  0.1770, -0.1901],
        [ 0.6272,  0.6980, -0.4561, -0.5890],
        [ 0.3884,  0.6853, -0.5808, -0.4200],
        [ 0.1225, -0.5776, -0.6393, -0.3510],
        [ 0.0936,  0.0701,  0.3228,  0.6653],
        [ 0.4604,  0.3984, -0.6797, -0.1293]])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [19]:
#5.2.2.2. 自定义初始化
def my_init(m):
    if type(m) == nn.Linear:
        print("Init",*[(name,param.shape) for name,param in m.named_parameters()][0])
        nn.init.uniform_(m.weight,-10,10)
        m.weight.data *= m.weight.data.abs() >= 5
net.apply(my_init)
net[0].weight

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


Parameter containing:
tensor([[-0.0000, -8.3835,  0.0000, -0.0000],
        [-6.0539,  9.2093, -9.1482, -6.1750],
        [-8.6153,  0.0000, -6.3991,  6.5672],
        [-0.0000, -0.0000, -8.8401,  0.0000],
        [ 0.0000, -0.0000,  0.0000,  0.0000],
        [-9.3192,  0.0000,  9.1029,  0.0000],
        [-9.9707, -0.0000, -6.1985,  0.0000],
        [ 0.0000, -9.8143,  9.9310, -0.0000]], requires_grad=True)

In [20]:
#直接设置参数
net[0].weight.data[:] += 1
net[0].weight.data[0,0] = 42
net[0].weight.data[0]

tensor([42.0000, -7.3835,  1.0000,  1.0000])

In [21]:
# 5.2.3. 参数绑定
shared = nn.Linear(8,8)
net = nn.Sequential(nn.Linear(4,8)
                    ,nn.ReLU()
                    ,shared
                    ,nn.ReLU()
                    ,shared
                    ,nn.Linear(8,1))
net(x)

tensor([[-0.1483],
        [-0.1147]], grad_fn=<AddmmBackward0>)

In [23]:
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0,0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
